# Setting up resources to quantify transcripts with Salmon on GCP
## PART2: run quantification

https://salmon.readthedocs.io/en/latest/salmon.html


#### Jan 07, 2020

In [75]:
# setup notebook global vars
import pandas as pd

#gcp 
project_id = 'nih-nia-lng-cbg'
prj_bucket = 'gs://nihnialngcbg-eqtl'
my_user='mooreank'
data = 'nabec'
reference = 'chess'
docker_img = 'us.gcr.io/nih-nia-lng-cbg/nihnialngcbg-salmon:2020-1'
WRKDIR = '/Users/mooreank/Desktop/Raph/requant/salmon'

#### Generate list of samples (change directories if necessary)

In [13]:
##get full sample list to run from fastqs

#gsutil ls gs://nihnialngcbg-eqtl/rosmap/fastqs/*_R1.fastq.gz > bucket.fastqs.list
!gsutil ls gs://nihnialngcbg-eqtl/nabec/totalrna/fastqs/*_R2.fastq.gz > /Users/mooreank/Desktop/Raph/requant/salmon/nabec.samples.list
#sed -i s"/_R1\.fastq\.gz//"g bucket.fastqs.list
!sed -i -e s"/_R2\.fastq\.gz//"g /Users/mooreank/Desktop/Raph/requant/salmon/nabec.samples.list
!sed -i -e s"/gs\:\/\/nihnialngcbg-eqtl\/nabec\/totalrna\/fastqs\///"g /Users/mooreank/Desktop/Raph/requant/salmon/nabec.samples.list

#!head -10 rosmap.samples.list > rosmap.samples.list.test


In [ ]:
###get list of samples in python 
sample_file = f'{WRKDIR}/{data}.samples.list'

samples_list= []
with open(sample_file, 'r') as fi:
    for line in fi:
        line = line.strip()
        samples_list.append(str(line))


In [87]:
len(samples_list)

339

In [23]:
#test = samples_list[:2]

#### Run requantification

In [ ]:
###for chess reference

# run a test of quantifying an RNA sample
# in_bucket = 'gs://nihnialngcbg-eqtl/rosmap'
# out_bucket = 'gs://nihnialngcbg-eqtl/rosmap_test'
# sample = ''
# project_id = 'nih-nia-lng-cbg'
# prj_bucket = 'gs://nihnialngcbg-eqtl'
# docker_img = 'us.gcr.io/nih-nia-lng-cbg/nihnialngcbg-salmon:2020-1'
# my_user = 'mooreank'



# gcp_lsci_pipe_cmd = 'mkdir -p /gcloud-shared/reference; \
# gsutil -mq cp ${SALMONTRANSCRIPTMODELS}/* /gcloud-shared/reference; \
# salmon --no-version-check quant -i /gcloud-shared/reference -l A -1 ${FASTQR1} -2 ${FASTQR2} \
# --validateMappings --seqBias --gcBias --posBias --threads $(nproc) \
# -o /gcloud-shared/sample; \
# gsutil -mq cp -r /gcloud-shared/sample ${GS_OUTPATH}; \
# ls -lhR /gcloud-shared'

# gcp_lsci_pipe_vars = f'SALMONTRANSCRIPTMODELS={prj_bucket}/resources/references/\
# salmon/chess/reference,\
# GS_OUTPATH={out_bucket}/quants/chess/{sample},\
# FASTQR1=/gcloud-shared/inputR1.gz,\
# FASTQR2=/gcloud-shared/inputR2.gz'

# cloud_cmd = f'gcloud beta lifesciences pipelines run \
# --project {project_id} \
# --docker-image {docker_img} \
# --machine-type n1-standard-4 \
# --logging {out_bucket}/logs/salmon/{sample} \
# --command-line \'{gcp_lsci_pipe_cmd}\' \
# --inputs FASTQR1={in_bucket}/fastqs/${{sample}}_R1.fastq.gz \
# --inputs FASTQR2={in_bucket}/fastqs/${{sample}}R2.fastq.gz \
# --env-vars {gcp_lsci_pipe_vars} \
# --labels=pipe=salmonquant,sample={sample},user={my_user} \
# --preemptible ' 

# print(f'{cloud_cmd}\n')

# #!{cloud_cmd}



In [99]:
###WORKS ROSMAP

def requant_commands(sample_list, out_file_name, in_bucket, out_bucket):
    out_file = open(f'{out_file_name}', "w")
    for sample in sample_list:        
    
        gcp_lsci_pipe_cmd = 'mkdir -p /gcloud-shared/reference; \
gsutil -mq cp ${SALMONTRANSCRIPTMODELS}/* /gcloud-shared/reference; \
salmon --no-version-check quant -i /gcloud-shared/reference -l A -1 ${FASTQR1} -2 ${FASTQR2} \
--validateMappings --seqBias --gcBias --posBias --threads $(nproc) -o /gcloud-shared/sample; \
gsutil -mq cp -r /gcloud-shared/sample ${GS_OUTPATH}; \
ls -lhR /gcloud-shared'

        gcp_lsci_pipe_vars = f'SALMONTRANSCRIPTMODELS={prj_bucket}/resources/references/salmon/{reference},\
GS_OUTPATH={out_bucket}/quants/{reference}/{sample},\
FASTQR1=/gcloud-shared/inputR1.gz,\
FASTQR2=/gcloud-shared/inputR2.gz'

        cloud_cmd = f'gcloud beta lifesciences pipelines run \
--project {project_id} \
--docker-image {docker_img} \
--machine-type n1-standard-4 \
--logging {out_bucket}/logs/salmon/{sample}/ \
--command-line \'{gcp_lsci_pipe_cmd}\' \
--inputs FASTQR1={in_bucket}/fastqs/{sample}_R1.fastq.gz \
--inputs FASTQR2={in_bucket}/fastqs/{sample}_R2.fastq.gz \
--env-vars {gcp_lsci_pipe_vars} \
--boot-disk-size 20 \
--labels=pipe=salmonquant,sample={sample},user={my_user}'
        label = f'{sample} OPID='
        print(f'echo -n {label}', file=out_file)
        print(f'{cloud_cmd}\n', file=out_file)
#         print(f'echo -n "{sample} OPID="', file=out_file)
#         print(f'{cloud_cmd}\n', file=out_file)


In [227]:
###WORKS NABEC    

def requant_commands_nabec(sample_list, out_file_name, in_bucket, out_bucket):
    out_file = open(f'{out_file_name}', "w")
    for sample in sample_list:        
        mod_sample = sample.lower()
        gcp_lsci_pipe_cmd = 'mkdir -p /gcloud-shared/reference; \
gsutil -mq cp ${SALMONTRANSCRIPTMODELS}/* /gcloud-shared/reference; \
salmon --no-version-check quant -i /gcloud-shared/reference -l A -1 ${FASTQR1} -2 ${FASTQR2} \
--validateMappings --seqBias --gcBias --posBias --threads $(nproc) -o /gcloud-shared/sample; \
gsutil -mq cp -r /gcloud-shared/sample ${GS_OUTPATH}; \
ls -lhR /gcloud-shared'

        gcp_lsci_pipe_vars = f'SALMONTRANSCRIPTMODELS={prj_bucket}/resources/references/salmon/{reference},\
GS_OUTPATH={out_bucket}/quants/{reference}/{sample},\
FASTQR1=/gcloud-shared/inputR1.gz,\
FASTQR2=/gcloud-shared/inputR2.gz'

        cloud_cmd = f'gcloud beta lifesciences pipelines run \
--project {project_id} \
--docker-image {docker_img} \
--machine-type n1-standard-4 \
--logging {out_bucket}/logs/salmon/{sample}/ \
--command-line \'{gcp_lsci_pipe_cmd}\' \
--inputs FASTQR1={in_bucket}/totalrna/fastqs/{sample}_R1.fastq.gz \
--inputs FASTQR2={in_bucket}/totalrna/fastqs/{sample}_R2.fastq.gz \
--env-vars {gcp_lsci_pipe_vars} \
--boot-disk-size 50 \
--labels=pipe=salmonquant,sample={mod_sample},user={my_user}'
        label = f'{sample} OPID='
        print(f'echo -n {label}', file=out_file)
        print(f'{cloud_cmd}\n', file=out_file)
#         print(f'echo -n "{sample} OPID="', file=out_file)
#         print(f'{cloud_cmd}\n', file=out_file)
#--preemptible


In [39]:

#Run requant with list of samples

WRKDIR = '/Users/mooreank/Desktop/Raph/requant/salmon'
reference = 'gencode_32_filtered'
data = 'nabec'
in_bucket = f'gs://nihnialngcbg-eqtl/{data}'
out_bucket = f'gs://nihnialngcbg-eqtl/{data}'           
file_name = f'{WRKDIR}/{data}_requant_commands_{reference}.sh'


#run requant function
#requant_commands(samples_list,file_name, in_bucket, out_bucket)
requant_commands_nabec(samples_list,file_name, in_bucket, out_bucket)



In [ ]:
###filtered 32 gencode x nabec (run 2/18/2020 1:30pm)
#run requant commands script in terminal
term_cmd = f'bash {file_name}'
!{term_cmd}

In [ ]:
###chess, nabec (run 2/13/2020 10:12am)

##run requant commands script in terminal
term_cmd = f'bash {file_name}'
!{term_cmd}

#### Check for requant fails

In [119]:
#get number of quants in out folder

data= 'nabec'
reference = 'gencode_32_filtered'
check = f'gsutil ls {prj_bucket}/{data}/quants/{reference}/*/quant.sf | wc -l'
print(check)
!{check}

gsutil ls gs://nihnialngcbg-eqtl/nabec/quants/gencode_32_filtered/*/quant.sf | wc -l
     311


In [137]:
#need to change directories
#get a list of all input fastqs again
# !gsutil ls gs://nihnialngcbg-eqtl/rosmap/fastqs/*_R2.fastq.gz > /Users/mooreank/Desktop/Raph/requant/salmon/rosmap.samples.list


# !sed -i s"/gs:\/\/nihnialngcgb-eqtl\/rosmap\/fastqs\///"g /Users/mooreank/Desktop/Raph/requant/salmon/rosmap.samples.list
# !sed -i -e s"/_R2\.fastq\.gz//"g /Users/mooreank/Desktop/Raph/requant/salmon/rosmap.samples.list
# !sed -i -e s"/gs\:\/\/nihnialngcbg-eqtl\/rosmap\/fastqs\///"g /Users/mooreank/Desktop/Raph/requant/salmon/rosmap.samples.list

#!head -10 rosmap.samples.list > rosmap.samples.list.test
    
print("Input fastqs:")
#!less /Users/mooreank/Desktop/Raph/requant/salmon/rosmap.samples.list | wc -l
cmd = f'less /Users/mooreank/Desktop/Raph/requant/salmon/{data}.samples.list | wc -l'
!{cmd}


Input fastqs:
     311


In [225]:
#get a list of files that were successfully created
##need to manually change reference path
reference = 'chess'
cmd = f'gsutil -mq ls gs://nihnialngcbg-eqtl/{data}/quants/{reference}/*/quant.sf > /Users/mooreank/Desktop/Raph/requant/salmon/{data}.found.files'
!{cmd}
cmd2 = f'sed -i -e s"/gs:\/\/nihnialngcbg-eqtl\/{data}\/quants\/{reference}\///"g /Users/mooreank/Desktop/Raph/requant/salmon/{data}.found.files'
!{cmd2}
cmd3 = f'sed -i -e s"/\/quant.sf//"g /Users/mooreank/Desktop/Raph/requant/salmon/{data}.found.files'
!{cmd3}

print("Quant files created:")
cmd4 = f'less /Users/mooreank/Desktop/Raph/requant/salmon/{data}.found.files | wc -l'
!{cmd4}

Quant files created:
     308


In [230]:
#check for any missing expected quant files

data = 'rosmap'
WRKDIR = '/Users/mooreank/Desktop/Raph/requant/salmon'
#expected_file = f'{WRKDIR}/{data}.samples.list'
expected_file = f'{WRKDIR}/{data}.samples.list'
observed_file = f'{WRKDIR}/{data}.found.files'
missing_file = f'{WRKDIR}/{data}.missing.samples.list'

expected = pd.read_csv(expected_file,header=None)
observed = pd.read_csv(observed_file,header=None)

print("Input fastqs:")
print(expected.shape)
print("Created quants:")
print(observed.shape)

print("Missing samples:")
print(len(set(expected[0]) - set(observed[0])))

missing = expected.loc[~expected[0].isin(observed[0])]
missing_samples = missing[0].tolist()
print(missing.shape)

#save the missing list
missing.to_csv(missing_file,header=None,index=None)

print('')
print("Missing sample IDs:")
print(missing_samples)

Input fastqs:
(339, 1)
Created quants:
(337, 1)
Missing samples:
2
(2, 1)

Missing sample IDs:
['191_120424', '380_120503']


In [110]:
reference

'chess'

#### Rerun requant with list of missing samples

In [231]:
in_bucket = f'gs://nihnialngcbg-eqtl/{data}'
out_bucket = f'gs://nihnialngcbg-eqtl/{data}'  
file_name = f'{WRKDIR}/{data}_requant_commands_missing_{reference}.sh'

#run requant function
requant_commands(missing_samples,file_name, in_bucket, out_bucket)
#requant_commands_nabec(missing_samples,file_name, in_bucket, out_bucket)

In [ ]:
##for nabec x chess run 2/21/2020 3:10pm
##run requant commands script in terminal
term_cmd = f'bash {file_name}'
!{term_cmd}

In [ ]:
# ##for gencode_32_filtered run 2/10/2020 2:56pm
# ##run requant commands script in terminal
# term_cmd = f'bash {file_name}'
# !{term_cmd}

In [ ]:
##for chess x rosmap run 2/19/2020 3:00pm
##run requant commands script in terminal
term_cmd = f'bash {file_name}'
!{term_cmd}

### check job status, id OPID above

In [ ]:

op_id = 2765249889847837074

!gcloud beta lifesciences operations describe {op_id}

In [ ]:
##make to txt file
# sample_list = samples
# gcloud_quants_dir = '/gcloud-shared/quants'
# sample_dir_file = "sample_gcloud_directories.txt"
# out_file = open(f'{sample_dir_file}', "w")    
# for x in sample_list:    
#     print((f'{gcloud_quants_dir}/'+x), file=out_file)
#     print(f'{gcloud_quants_dir}/'+x)

#### Run quantmerge on all quant files

In [219]:
###get list of samples in python 
data = 'nabec'
WRKDIR = '/Users/mooreank/Desktop/Raph/requant/salmon'
sample_file = f'{WRKDIR}/{data}.samples.list'

samples_list= []
with open(sample_file, 'r') as fi:
    for line in fi:
        line = line.strip()
        samples_list.append(str(line))


## Rename samples

In [ ]:
##rna to dna names (rosmap)
#gs://nihnialngcbg-eqtl/rosmap/rna.to.dna.names.map
data = 'nabec'
WRKDIR = '/Users/mooreank/Desktop/Raph/requant'
dna_names = pd.read_csv(f'{WRKDIR}/{data}/rna.to.dna.names.map', sep='\t', header=None, index_col=False)
#dna_names = dna_names.set_index(0)
#dna_names = dna_names.reindex(samples_list)
rna_samples = dna_names[0].tolist()
print(len(rna_samples))
dna_samples = dna_names[1].tolist()
print(len(dna_samples))

print(dna_names.shape)
dna_names.head()

In [123]:
rna_test = rna_samples[:10]
dna_test = dna_samples[:10]

In [212]:
# #no new names
# def quantmerge_command(out_bucket, in_bucket, out_file_name, sample_list, column_type, quant_type):
#     gcloud_quants_dir = '/gcloud-shared/quants'
#     sample_string = ''
#     for x in sample_list:
#         sample_string = sample_string+f'{gcloud_quants_dir}/'+x+' '
#     if quant_type == 'gene':
#         gene = '--genes '
#     else:
#         gene = ''
#     if column_type == 'tpm':
#         column_type = ''
#     if column_type == 'numreads':
#         column_type = '--column numreads '
    
#     gcp_lsci_pipe_cmd = f'mkdir -p /gcloud-shared/quants; \
# mkdir -p /gcloud-shared/merge; \
# gsutil -mq cp -r ${{QUANTS}}/* /gcloud-shared/quants/; \
# salmon --no-version-check quantmerge --quants {sample_string}{column_type}{gene}-o /gcloud-shared/merge/{out_file_name}; \
# gsutil -mq cp /gcloud-shared/merge/* ${{GS_OUTPATH}}; \
# ls -lhR /gcloud-shared'

#     gcp_lsci_pipe_vars = f'QUANTS={in_bucket}/quants/{reference},\
# GS_OUTPATH={out_bucket}/quantmerge/{reference}/'
        
#     cloud_cmd = f'gcloud beta lifesciences pipelines run \
# --project {project_id} \
# --docker-image {docker_img} \
# --machine-type n1-standard-4 \
# --logging {out_bucket}/logs/salmon_merge/{reference}_merge.log \
# --command-line \'{gcp_lsci_pipe_cmd}\' \
# --env-vars {gcp_lsci_pipe_vars} \
# --boot-disk-size 60 \
# --labels=pipe=salmonquantmerge,user={my_user}'
#     #--preemptible'
#     #print(f'echo -n "{sample} OPID="', file=out_file)
#     #print(f'{cloud_cmd}\n', file=out_file)
#     #print(f'echo -n "{sample} OPID="')
#     print(f'{cloud_cmd}\n')
#     !{cloud_cmd}

    

## Merge samples with new sample names

In [216]:
##with new sample names 
def quantmerge_command_names(in_bucket, out_bucket, out_file_name, sample_list, dna_names, column_type, quant_type):
    gcloud_quants_dir = '/gcloud-shared/quants'
    sample_string = ''
    for x in sample_list:
        sample_string = sample_string+f'{gcloud_quants_dir}/'+x+' '
    dna_string = ''
    for x in dna_names:
        dna_string = dna_string+x+' '
    if quant_type == 'gene':
        gene = '--genes '
    else:
        gene = ''
    if column_type == 'tpm':
        column_type = ''
    if column_type == 'numreads':
        column_type = '--column numreads '
    
    gcp_lsci_pipe_cmd = f'mkdir -p /gcloud-shared/quants; \
mkdir -p /gcloud-shared/merge; \
gsutil -mq cp -r ${{QUANTS}}/* /gcloud-shared/quants/; \
salmon --no-version-check quantmerge --quants {sample_string}--names {dna_string}{column_type}{gene}-o /gcloud-shared/merge/{out_file_name}; \
gsutil -mq cp /gcloud-shared/merge/{out_file_name} ${{GS_OUTPATH}}; \
ls -lhR /gcloud-shared'

    gcp_lsci_pipe_vars = f'QUANTS={in_bucket}/quants/{reference},\
GS_OUTPATH={out_bucket}/quantmerge/{reference}/'
        
    cloud_cmd = f'gcloud beta lifesciences pipelines run \
--project {project_id} \
--docker-image {docker_img} \
--machine-type n1-standard-4 \
--logging {out_bucket}/logs/salmon/quantmerge \
--command-line \'{gcp_lsci_pipe_cmd}\' \
--env-vars {gcp_lsci_pipe_vars} \
--boot-disk-size 20 \
--labels=pipe=salmonquantmerge,user={my_user}'
    #--preemptible'
    #print(f'echo -n "{sample} OPID="', file=out_file)
    #print(f'{cloud_cmd}\n', file=out_file)
    #print(f'echo -n "{sample} OPID="')
    print(f'{cloud_cmd}\n')
    !{cloud_cmd}

    

In [167]:
len(samples_list)

339

In [ ]:
##change column_type for tpms vs numreads and quant_type for transcript vs gene quant

data = 'nabec'
reference = 'gencode_32_filtered'
#column_type = 'tpm'
column_type = 'numreads'
#quant_type = 'transcript'
quant_type = 'gene'


in_bucket = f'gs://nihnialngcbg-eqtl/{data}'
out_bucket = f'gs://nihnialngcbg-eqtl/{data}'
out_file_name = f'{data}_{quant_type}_{column_type}.sf'



quantmerge_command_names(in_bucket, out_bucket, out_file_name, rna_samples, dna_samples, column_type, quant_type)
#quantmerge_command(in_bucket, out_bucket, out_file_name, samples_list, column_type, quant_type)
#quantmerge_command_names_test(in_bucket, out_bucket, out_file_name, rna_samples, dna_samples)

In [ ]:
#check job status, id OPID above
op_id = 12446396281431860167

!gcloud beta lifesciences operations describe {op_id}

In [74]:
#see how many GCE (Google Compute Engine) instances are running your jobs
PIPELABEL='salmonquantmerge'
PROJECT_ID='nih-nia-lng-cbg'
COHORT='rosmap'
MYUSER='mooreank'

print('#full job worker node count')
cmd = f'gcloud compute instances list --project {PROJECT_ID} \
--filter "labels:({PIPELABEL} {COHORT} {MYUSER})" | grep RUNNING | wc -l'
!{cmd}

print('#number of all running instances in project')
cmd2 = f'gcloud compute instances list --project {PROJECT_ID} | grep RUNNING | wc -l'
!{cmd2}

#full job worker node count
       9
#number of all running instances in project
      27
